In [1]:
# Install and import necessary libraries
!pip install gradio
!pip install pulp

import pandas as pd
import os
import numpy as np
import gradio as gr
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 15.9 MB/s eta 0:00:00


In [2]:
# Import data
file_name = 'Baseballdata.csv'

In [3]:
# Read in file_name into pandas DataFrame 'df'

baseball_stats_df = pd.read_csv(file_name)

In [4]:
# Load the dataset
baseball_stats_df = pd.read_csv(file_name)  # Read a CSV file into a DataFrame named 'baseball_stats_df'

# Ensure required columns are present
if "SB" not in baseball_stats_df.columns:
    # If the "SB" (stolen bases) column is missing, generate random values between 0 and 50
    baseball_stats_df["SB"] = np.random.randint(0, 50, size=len(baseball_stats_df))

# Define a function to calculate ratings for the 20-80 scale
def calculate_rating(value, min_value, max_value):
    """
    Convert a value into a 20-80 rating based on min and max.
    The 20-80 scale is commonly used in baseball scouting to rate player skills.

    Parameters:
        value (float): The value to convert to a 20-80 rating.
        min_value (float): The minimum value in the dataset for normalization.
        max_value (float): The maximum value in the dataset for normalization.

    Returns:
        int: The calculated rating on a 20-80 scale.
    """
    return int(np.clip(20 + (value - min_value) / (max_value - min_value) * 60, 20, 80))

# Calculate ratings for each player
def assign_player_ratings(data):
    """
    Assign 20-80 scale ratings to players based on various performance metrics.

    Parameters:
        data (DataFrame): The DataFrame containing player statistics.

    Returns:
        DataFrame: The DataFrame with added rating columns.
    """
    # Initialize a dictionary to hold ratings for each skill
    ratings = {
        "Hitting": [],
        "Hitting_for_Power": [],
        "Fielding": [],
        "Throwing": [],
        "Speed": [],
    }

    # Loop through each skill and calculate its composite rating
    for col, values in ratings.items():
        if col == "Hitting":
            # Hitting skill combines batting average (AVG) and on-base percentage (OBP)
            avg = data["AVG"]
            obp = data["OBP"]
            composite = avg + obp  # Composite value is the sum of AVG and OBP
            min_value, max_value = composite.min(), composite.max()
            # Calculate ratings using the composite value
            ratings[col] = [calculate_rating(v, min_value, max_value) for v in composite]

        elif col == "Hitting_for_Power":
            # Power hitting combines slugging percentage (SLG), isolated power (ISO), and normalized home runs (HR)
            slg = data["SLG"]
            iso = data["ISO"]
            hr = data["HR"]
            composite = slg + iso + (hr / 50)  # Normalize HR to a small range
            min_value, max_value = composite.min(), composite.max()
            ratings[col] = [calculate_rating(v, min_value, max_value) for v in composite]

        elif col == "Fielding":
            # Fielding skill combines fielding percentage and range factor
            fielding = data["Fielding_Percentage"]
            range_ = data["Fielding_Range"]
            composite = fielding + range_ / 5  # Normalize range to match percentage scales
            min_value, max_value = composite.min(), composite.max()
            ratings[col] = [calculate_rating(v, min_value, max_value) for v in composite]

        elif col == "Throwing":
            # Throwing skill is based on throwing velocity
            throwing = data["Throwing_Velocity"]
            min_value, max_value = throwing.min(), throwing.max()
            ratings[col] = [calculate_rating(v, min_value, max_value) for v in throwing]

        elif col == "Speed":
            # Speed combines stolen bases (SB) and weighted fielding range
            sb = data["SB"]
            range_ = data["Fielding_Range"]
            composite = sb + range_ * 5  # Give fielding range a stronger weight for speed
            min_value, max_value = composite.min(), composite.max()
            ratings[col] = [calculate_rating(v, min_value, max_value) for v in composite]

    # Add the calculated ratings as new columns to the DataFrame
    for col, values in ratings.items():
        data[col] = values
    return data

# Assign ratings to players
baseball_stats_with_ratings = assign_player_ratings(baseball_stats_df)

In [5]:
# Define a chatbot for player comparison
def compare_players(player1_name, player2_name, dataset):
    """Compare two players across evaluation categories."""
    player1_data = dataset.loc[dataset["Player"] == player1_name]
    player2_data = dataset.loc[dataset["Player"] == player2_name]

    if player1_data.empty or player2_data.empty:
        missing_player = player1_name if player1_data.empty else player2_name
        return f"Player '{missing_player}' not found."

    categories = ["Hitting", "Hitting_for_Power", "Fielding", "Throwing", "Speed"]
    comparison = f"Comparison between {player1_name} and {player2_name}:\n"

    for category in categories:
        player1_score = player1_data[category].values[0]
        player2_score = player2_data[category].values[0]

In [6]:
# Compare two players by name
example_comparison = compare_players("Player 1", "Player 2", baseball_stats_with_ratings)
print(example_comparison)

None


In [7]:
print(baseball_stats_with_ratings["Player"].unique())

['Player 1' 'Player 2' 'Player 3' 'Player 4' 'Player 5' 'Player 6'
 'Player 7' 'Player 8' 'Player 9' 'Player 10' 'Player 11' 'Player 12'
 'Player 13' 'Player 14' 'Player 15' 'Player 16' 'Player 17' 'Player 18'
 'Player 19' 'Player 20' 'Player 21' 'Player 22' 'Player 23' 'Player 24'
 'Player 25' 'Player 26' 'Player 27' 'Player 28' 'Player 29' 'Player 30'
 'Player 31' 'Player 32' 'Player 33' 'Player 34' 'Player 35' 'Player 36'
 'Player 37' 'Player 38' 'Player 39' 'Player 40' 'Player 41' 'Player 42'
 'Player 43' 'Player 44' 'Player 45' 'Player 46' 'Player 47' 'Player 48'
 'Player 49' 'Player 50' 'Player 51' 'Player 52' 'Player 53' 'Player 54'
 'Player 55' 'Player 56' 'Player 57' 'Player 58' 'Player 59' 'Player 60'
 'Player 61' 'Player 62' 'Player 63' 'Player 64' 'Player 65' 'Player 66'
 'Player 67' 'Player 68' 'Player 69' 'Player 70' 'Player 71' 'Player 72'
 'Player 73' 'Player 74' 'Player 75' 'Player 76' 'Player 77' 'Player 78'
 'Player 79' 'Player 80' 'Player 81' 'Player 82' 'Player 83'

In [16]:
def compare_players(player1_name, player2_name, dataset):
    """Compare two players and produce a formatted output."""
    # Retrieve player data
    player1_data = dataset.loc[dataset["Player"] == player1_name]
    player2_data = dataset.loc[dataset["Player"] == player2_name]

    # Check if both players exist
    if player1_data.empty or player2_data.empty:
        missing_player = player1_name if player1_data.empty else player2_name
        return f"Player '{missing_player}' not found in the dataset."

    # Categories for comparison
    categories = ["Hitting", "Hitting_for_Power", "Fielding", "Throwing", "Speed"]
    comparison = f"🆚 **Comparison: {player1_name} vs {player2_name}**\n"
    comparison += "-" * 40 + "\n"

    # Individual category comparison
    for category in categories:
        player1_score = player1_data[category].values[0]
        player2_score = player2_data[category].values[0]
        winner = (
            player1_name if player1_score > player2_score else
            player2_name if player2_score > player1_score else
            "Tie"
        )
        comparison += f"🏆 **{category}**\n"
        comparison += f"   {player1_name}: {player1_score}/80\n"
        comparison += f"   {player2_name}: {player2_score}/80\n"
        comparison += f"   Winner: {winner}\n\n"

    # Overall comparison - Average of five categories rescaled to 20-80
    player1_average = player1_data[categories].mean(axis=1).values[0]
    player2_average = player2_data[categories].mean(axis=1).values[0]

    # Rescale averages to 20-80 scale
    def rescale_to_20_80(score):
        return 20 + (score / 80) * 60

    player1_rescaled = rescale_to_20_80(player1_average)
    player2_rescaled = rescale_to_20_80(player2_average)

    overall_winner = (
        player1_name if player1_rescaled > player2_rescaled else
        player2_name if player2_rescaled > player1_rescaled else
        "Tie"
    )

    comparison += "-" * 40 + "\n"
    comparison += f"🌟 **Overall Winner:** {overall_winner}\n"
    comparison += f"   {player1_name} Overall: {player1_rescaled:.1f}/80\n"
    comparison += f"   {player2_name} Overall: {player2_rescaled:.1f}/80\n"
    comparison += "-" * 40

    return comparison


In [21]:
print(compare_players("Player 49", "Player 356", baseball_stats_with_ratings))

🆚 **Comparison: Player 49 vs Player 356**
----------------------------------------
🏆 **Hitting**
   Player 49: 63/80
   Player 356: 72/80
   Winner: Player 356

🏆 **Hitting_for_Power**
   Player 49: 32/80
   Player 356: 36/80
   Winner: Player 356

🏆 **Fielding**
   Player 49: 44/80
   Player 356: 38/80
   Winner: Player 49

🏆 **Throwing**
   Player 49: 20/80
   Player 356: 57/80
   Winner: Player 356

🏆 **Speed**
   Player 49: 54/80
   Player 356: 26/80
   Winner: Player 49

----------------------------------------
🌟 **Overall Winner:** Player 356
   Player 49 Overall: 52.0/80
   Player 356 Overall: 54.4/80
----------------------------------------


In [10]:
# Load the full dataset
dataset_path = "Baseballdata.csv"
baseball_stats_df = pd.read_csv(dataset_path)

# Ensure the "Player" column exists and is unique
if "Player" not in baseball_stats_df.columns:
    raise ValueError("The dataset does not contain a 'Player' column.")
if baseball_stats_df["Player"].duplicated().any():
    baseball_stats_df["Player"] = baseball_stats_df["Player"] + "_" + baseball_stats_df.index.astype(str)

# The evaluation function
def evaluate_player(player_name):
    """
    Evaluate a selected player's skills based on their statistics.
    """
    # Get the player's stats from the dataset
    player_stats = baseball_stats_df[baseball_stats_df["Player"] == player_name].iloc[0]

    # Extract statistics
    avg = player_stats["AVG"]
    obp = player_stats["OBP"]
    slg = player_stats["SLG"]
    iso = player_stats["ISO"]
    hr = player_stats["HR"]
    fielding_pct = player_stats["Fielding_Percentage"]
    fielding_range = player_stats["Fielding_Range"]
    throwing_velocity = player_stats["Throwing_Velocity"]
    sb = player_stats["SB"]

    # Calculate composite metrics for each skill
    hitting = avg + obp
    hitting_for_power = slg + iso + (hr / 50)
    fielding = fielding_pct + fielding_range / 5
    throwing = throwing_velocity
    speed = sb + fielding_range * 5

    # Normalize the 20-80 scale based on typical baseball data ranges
    def calculate_rating(value, min_value, max_value):
        return int(np.clip(20 + (value - min_value) / (max_value - min_value) * 60, 20, 80))

    ratings = {
        "Hitting": calculate_rating(hitting, 0.4, 1.5),
        "Hitting_for_Power": calculate_rating(hitting_for_power, 0.5, 2.5),
        "Fielding": calculate_rating(fielding, 0.9, 1.2),
        "Throwing": calculate_rating(throwing, 70, 100),
        "Speed": calculate_rating(speed, 0, 300),
    }

    # Generate a comprehensive evaluation
    evaluation = []

    if ratings["Hitting"] >= 70:
        evaluation.append("Elite hitter with exceptional consistency and plate discipline.")
    elif ratings["Hitting"] >= 50:
        evaluation.append("Solid hitter with above-average consistency.")
    else:
        evaluation.append("Needs improvement in hitting to enhance overall performance.")

    if ratings["Hitting_for_Power"] >= 70:
        evaluation.append("Impressive power hitter capable of hitting many home runs.")
    elif ratings["Hitting_for_Power"] >= 50:
        evaluation.append("Shows occasional power but not a primary strength.")
    else:
        evaluation.append("Limited power-hitting ability, which impacts offensive value.")

    if ratings["Fielding"] >= 70:
        evaluation.append("Outstanding fielder with excellent defensive skills and reliability.")
    elif ratings["Fielding"] >= 50:
        evaluation.append("Dependable fielder with average range and ability.")
    else:
        evaluation.append("Defensive skills need significant improvement to reduce errors.")

    if ratings["Throwing"] >= 70:
        evaluation.append("Top-tier arm strength, capable of making tough throws with ease.")
    elif ratings["Throwing"] >= 50:
        evaluation.append("Adequate arm strength for most defensive plays.")
    else:
        evaluation.append("Limited throwing strength, which may restrict defensive versatility.")

    if ratings["Speed"] >= 70:
        evaluation.append("Exceptional speed, a significant threat on the base paths.")
    elif ratings["Speed"] >= 50:
        evaluation.append("Average speed, capable of taking extra bases when needed.")
    else:
        evaluation.append("Below-average speed, which limits base-stealing potential.")

    # Combine the evaluation into a single string
    return f"Evaluation for {player_name}: " + " ".join(evaluation)


# Create a Gradio interface
app = gr.Interface(
    fn=evaluate_player,
    inputs=gr.Dropdown(choices=baseball_stats_df["Player"].tolist(), label="Select a Player"),
    outputs=gr.Textbox(label="Player Evaluation"),
    title="Baseball Player Evaluation",
    description="Select a player from the dropdown to receive a detailed evaluation of their skills based on the 20-80 scouting scale."
)

# Launch the Gradio app
app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b1a736689f9727e3d0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [12]:
# Use pulp library to optimize team of thirteen hitters while adhering to a given maximum total salary
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Load the dataset
dataset_path = "Baseballdata.csv"  # Replace with the correct file path
baseball_stats_df = pd.read_csv(dataset_path)

def optimize_team_with_pulp(baseball_stats_df, max_salary=300000000, min_players=13, max_players_per_position=2):
    """
    Optimize the selection of players to maximize performance while staying as close to the salary cap as possible,
    enforcing position constraints with at least one player per position and at most two players per position.
    """
    # Add a performance score for each player
    baseball_stats_df["Performance"] = (
        baseball_stats_df["WRC+"] +  # Offensive performance
        baseball_stats_df["OPS+"] +  # Hitting effectiveness
        baseball_stats_df["Fielding_Range"] +  # Defensive range
        baseball_stats_df["SB"] +  # Speed and baserunning
        baseball_stats_df["Throwing_Velocity"]  # Arm strength
    )

    # Extract relevant data for optimization
    salaries = baseball_stats_df["Salary"].values  # Salaries of players
    performance = baseball_stats_df["Performance"].values  # Performance scores of players
    positions = baseball_stats_df["Position"].values  # Positions of players
    players = baseball_stats_df["Player"].values  # Player names

    # Define the optimization problem
    prob = LpProblem("Team_Selection", LpMaximize)

    # Define decision variables for each player (binary: 1 = selected, 0 = not selected)
    x = [LpVariable(f"x_{i}", cat="Binary") for i in range(len(players))]

    # Objective: maximize total team performance
    prob += lpSum(x[i] * performance[i] for i in range(len(players))), "Total_Performance"

    # Constraint 1: Total salary must not exceed the salary cap
    prob += lpSum(x[i] * salaries[i] for i in range(len(players))) <= max_salary, "Total_Salary_Cap"

    # Constraint 2: Exactly `min_players` must be selected
    prob += lpSum(x[i] for i in range(len(players))) == min_players, "Total_Players"

    # Constraint 3: At least one player per position
    unique_positions = baseball_stats_df["Position"].unique()
    for position in unique_positions:
        prob += lpSum(x[i] for i in range(len(players)) if positions[i] == position) >= 1, f"Min_{position}_Players"

    # Constraint 4: At most `max_players_per_position` players per position
    for position in unique_positions:
        prob += lpSum(x[i] for i in range(len(players)) if positions[i] == position) <= max_players_per_position, f"Max_{position}_Players"

    # Solve the optimization problem
    prob.solve()

    # Extract the selected players based on the solution
    selected_indices = [i for i in range(len(players)) if x[i].varValue == 1]
    selected_players = baseball_stats_df.iloc[selected_indices]

    # Calculate total salary and performance
    total_salary = selected_players["Salary"].sum()
    total_performance = selected_players["Performance"].sum()

    # Print summary of results
    print(f"Total Salary: ${total_salary}")
    print(f"Total Performance Score: {total_performance}")
    return selected_players

# Optimize the team
optimal_team = optimize_team_with_pulp(baseball_stats_df, max_salary=300000000, min_players=13, max_players_per_position=2)

# Print the selected players
print("Selected Players:")
for _, row in optimal_team.iterrows():
    print(f"{row['Position']}: {row['Player']} - WRC+: {row['WRC+']}, OPS+: {row['OPS+']}, Fielding_Range: {row['Fielding_Range']}, SB: {row['SB']}, Throwing_Velocity: {row['Throwing_Velocity']}, Salary: {row['Salary']}")

# Visualize the players on a baseball field
def print_field(players_df):
    print("\nBaseball Field Visualization:")
    field_positions = {
        "C": "Catcher",
        "1B": "First Base",
        "2B": "Second Base",
        "3B": "Third Base",
        "SS": "Shortstop",
        "LF": "Left Field",
        "CF": "Center Field",
        "RF": "Right Field"
    }
    for pos, label in field_positions.items():
        players_at_pos = players_df[players_df["Position"] == pos]
        if not players_at_pos.empty:
            for _, player in players_at_pos.iterrows():
                print(f"{label}: {player['Player']} (WRC+: {player['WRC+']}, OPS+: {player['OPS+']}, Fielding_Range: {player['Fielding_Range']}, SB: {player['SB']}, Throwing_Velocity: {player['Throwing_Velocity']})")

print_field(optimal_team)


Total Salary: $122957079.0
Total Performance Score: 6321.000000000001
Selected Players:
2B: Player 6 - WRC+: 174, OPS+: 184, Fielding_Range: 5.9, SB: 31, Throwing_Velocity: 91, Salary: 740000.0
LF: Player 34 - WRC+: 182, OPS+: 176, Fielding_Range: 4.9, SB: 52, Throwing_Velocity: 99, Salary: 740000.0
3B: Player 55 - WRC+: 141, OPS+: 193, Fielding_Range: 4.9, SB: 50, Throwing_Velocity: 93, Salary: 740000.0
RF: Player 114 - WRC+: 183, OPS+: 192, Fielding_Range: 5.1, SB: 13, Throwing_Velocity: 97, Salary: 740000.0
LF: Player 143 - WRC+: 191, OPS+: 184, Fielding_Range: 4.9, SB: 36, Throwing_Velocity: 98, Salary: 740000.0
1B: Player 147 - WRC+: 153, OPS+: 188, Fielding_Range: 4.1, SB: 8, Throwing_Velocity: 93, Salary: 740000.0
RF: Player 225 - WRC+: 167, OPS+: 174, Fielding_Range: 5.1, SB: 47, Throwing_Velocity: 91, Salary: 980059.0
CF: Player 242 - WRC+: 159, OPS+: 177, Fielding_Range: 6.3, SB: 47, Throwing_Velocity: 90, Salary: 26300000.0
3B: Player 249 - WRC+: 172, OPS+: 169, Fielding_Ran